In [1]:
import sys
import os

thesis_path = "/" + os.path.join(
    *os.getcwd().split(os.path.sep)[:-1])
sys.path.append(thesis_path)


In [8]:
from glob import glob
metrics_paths = list(
    glob(os.path.join(thesis_path, "evaluations", "metrics", "tmp",
                      r"*.pkl.tar.gz"),
         recursive=True))
metrics_paths

['/home/loebbert/projects/thesis/evaluations/metrics/tmp/c17952c1-143a-49d7-b16a-4e743b55f320.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/1a7d6723-7573-4b7e-a4aa-564b349b9b71.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/f880615d-2daa-4cd6-973f-ffa617c42c7e.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/385b2fec-8944-423d-959e-feed3603d948.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/115fce18-b656-4a33-b722-756dd437dfe2.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/e306080b-40a7-400f-92e3-400c0cf5a0a1.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/2397bd75-e0cc-434d-b6ab-0a155f97c064.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/2b42b22a-806c-4f17-a5b4-5ffcafc6b831.pkl.tar.gz',
 '/home/loebbert/projects/thesis/evaluations/metrics/tmp/9b1c4101-3d6d-431b-b9d0-a2e5b51dc8af.pkl.tar.gz',
 '/home/loebbert/projects/thesis/eval

In [3]:
from evaluations.content_02 import get_per_sample_metrics

metrics_df = get_per_sample_metrics()

/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidge

/home/loebbert/projects/thesis/experiments/02_content/data/seed_3/04_metrics/size_2000/error_ratio_10/error_data_train/best_error_search_error_train.pkl


/home/loebbert/miniconda3/envs/haystack/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


/home/loebbert/projects/thesis/experiments/02_content/data/seed_3/04_metrics/size_4000/error_ratio_15/error_data_gazetteer/best_full_search_dev.pkl
/home/loebbert/projects/thesis/experiments/02_content/data/seed_3/04_metrics/size_4000/error_ratio_15/error_data_gazetteer/last_full_search_dev.pkl


KeyboardInterrupt: 

In [4]:
import os

bad_files = ["/home/loebbert/projects/thesis/experiments/02_content/data/seed_3/04_metrics/size_2000/error_ratio_10/error_data_train/best_error_search_error_train.pkl", 
             "/home/loebbert/projects/thesis/experiments/02_content/data/seed_3/04_metrics/size_4000/error_ratio_15/error_data_gazetteer/best_full_search_dev.pkl",
                "/home/loebbert/projects/thesis/experiments/02_content/data/seed_3/04_metrics/size_4000/error_ratio_15/error_data_gazetteer/last_full_search_dev.pkl"]

for file_path in bad_files:
    os.remove(file_path)

In [ ]:

datasets: Dict[str, str] = {}
seeds = [1, 2, 3]
sizes = [2000, 4000, 6000, 8000]
error_ratios = [0, 5, 10, 15]
parts = ["train", "dev", "test"]

with open(
        os.path.join(thesis_path, "data", "mlowner",
                        "lowner_types.json")) as file:
    types = list(json.load(file)["entities"].keys())

for seed, size, error_ratio, part in product(seeds, sizes,
                                                error_ratios, parts):
    if part in ["train", "dev"]:
        datasets[f"{seed}_{size}_{error_ratio}_{part}"] = os.path.join(
            thesis_path, "experiments", "02_content", "data",
            f"seed_{seed}", "00_datasets", f"size_{size}",
            f"error_ratio_{error_ratio}", f"error_lowner_{part}.json")
    else:
        datasets[f"{seed}_{size}_{error_ratio}_{part}"] = os.path.join(
            thesis_path, "data", "mlowner", "lowner_test.json")

for metrics_filepath in bad_files:
    metrics_list = []
    fp = metrics_filepath.split(os.path.sep)
    with open(metrics_filepath, "rb") as file:
        metrics: ASPMetrics = pickle.load(file).to("cpu")
    name = os.path.splitext(fp[-1])[0].split("_")
    timestep = 0
    if name[1] == "sampled":
        timestep = 1
    elif name[1] == "full":
        timestep = 2
    seed = int(fp[-6][-1])
    gazetteer_size = int(fp[-4].split("_")[1])
    error_ratio = int(fp[-3].split("_")[2])
    error_part = fp[-2].split("_")[2]
    with open(datasets[f"{seed}_{gazetteer_size}_{error_ratio}_{name[-1]}"],
              "r") as file:
        dataset = json.load(file)
    metrics_per_sample = metrics.metrics_per_sample(
        dataset, types).to_dict(orient="records")
    for sample_metrics in metrics_per_sample:
        metrics_list.append({
            "seed": seed,
            "checkpoint": name[0],
            "gazetteer_size": gazetteer_size,
            "error_ratio": error_ratio,
            "error_part": error_part,
            "timestep": timestep,
            "dataset": "lowner_" + name[-1],
            **sample_metrics
        })
    if error_ratio == 0 and error_part == "both":
        for part in ["train", "gazetteer"]:
            for sample_metrics in metrics_per_sample:
                metrics_list.append({
                    "seed": seed,
                    "checkpoint": name[0],
                    "gazetteer_size": gazetteer_size,
                    "error_ratio": error_ratio,
                    "error_part": part,
                    "timestep": timestep,
                    "dataset": "lowner_" + name[-1],
                    **sample_metrics
                })
    # save to file
    metrics_df = pd.DataFrame.from_records(metrics_list)
    metrics_path = os.path.join(thesis_path, "evaluations", "metrics", "tmp",
                                str(uuid4()) + ".pkl.tar.gz")
    metrics_df.to_pickle(metrics_path)
    print(metrics_path)


In [10]:
metrics_df = pd.DataFrame.from_records(all_metrics)
metrics_df.head()

,seed,checkpoint,gazetteer_size,error_ratio,timestep,dataset,f1
0,2,best,8000,0,1,sampled_search_test,0.8624
1,2,best,8000,0,0,error_search_error_dev,0.8623
2,2,last,8000,0,2,full_search_dev,0.8749
3,2,last,8000,0,1,sampled_search_dev,0.8630
4,2,last,8000,0,0,error_search_test,0.8623


In [11]:
metrics_df.describe()

,seed,gazetteer_size,error_ratio,timestep,f1
count,864.00000,864.000000,864.000000,864.00000,864.000000
mean,2.00000,5000.000000,7.500000,1.00000,0.879439
std,0.81697,2237.363123,5.593408,0.81697,0.050166
min,1.00000,2000.000000,0.000000,0.00000,0.717500
25%,1.00000,3500.000000,3.750000,0.00000,0.851300
50%,2.00000,5000.000000,7.500000,1.00000,0.864450
75%,3.00000,6500.000000,11.250000,2.00000,0.924850
max,3.00000,8000.000000,15.000000,2.00000,0.973500


In [15]:
metrics_df.iloc[metrics_df[(metrics_df["timestep"]==2) & (metrics_df["dataset"].str.endswith("test"))]["f1"].idxmin()]

seed                             2
checkpoint                    last
gazetteer_size                4000
error_ratio                     15
timestep                         2
dataset           full_search_test
f1                           0.818
Name: 276, dtype: object